### Analysis on iOS and Android apps Data

For this project, we'll pretend we're working as
data analysts for a company that builds Android
and iOS mobile apps. We make our apps available
on Google Play and the App Store.

Our goal for this project is to analyze data to help
our developers understand what type of apps are likely
to attract more users on Google Play Store and App Store.

In [63]:
import csv
applestore_file = open('AppleStore.csv')
data_apple = csv.reader(applestore_file)
dataset_iOs = list(data_apple)

googlestore_file = open('googleplaystore.csv')
data_google = csv.reader(googlestore_file)
dataset_android = list(data_google)

In [64]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [65]:
number_rows_android = len(dataset_android)-1
number_rows_apple = len(dataset_iOs)-1

In [66]:
explore_data(dataset_android, 1, 4, rows_and_columns=False)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']




In [67]:
explore_data(dataset_iOs, 1, 4, rows_and_columns=False)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']




In [68]:
dataset_iOs[0]

['id',
 'track_name',
 'size_bytes',
 'currency',
 'price',
 'rating_count_tot',
 'rating_count_ver',
 'user_rating',
 'user_rating_ver',
 'ver',
 'cont_rating',
 'prime_genre',
 'sup_devices.num',
 'ipadSc_urls.num',
 'lang.num',
 'vpp_lic']

In this dataset, I think the following columns can
prove to be useful:
[track_name, size_bytes, currency, price, user_rating, cont_rating, prime_genre]

## Data Cleaning

We'll need to:

Remove non-English apps like 爱奇艺PPS -《欢乐颂2》电视剧热播.
Remove apps that aren't free.

Row 10472 excluding the dataset column row has error
and has a column shift

In [69]:
dataset_android[10473]

['Life Made WI-Fi Touchscreen Photo Frame',
 '1.9',
 '19',
 '3.0M',
 '1,000+',
 'Free',
 '0',
 'Everyone',
 '',
 'February 11, 2018',
 '1.0.19',
 '4.0 and up']

In [70]:
del dataset_android[10473]

Now we go about removing the duplicate entries

Some apps have been recorded more than once in the
dataset.

In [71]:
duplicate_apps = []
unique_apps = []

for app in dataset_android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))

Number of duplicate apps: 1181


We use the following criterion for keeping a record for
for an app.

We'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.

In [73]:
android_header = dataset_android[0]
del dataset_android[0]

In [83]:
ios_header = dataset_iOs[0]
del dataset_iOs[0]

In [74]:
reviews_max = {}

for app in dataset_android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

Check if the dataset has been properly cleaned by checking its new size after cleaning.


In [75]:
print('Expected length:', len(dataset_android) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


Nicely done !

Now, let's use the reviews_max dictionary to remove the
duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews.In the code cell below:

In [76]:
android_clean = []
already_added = []

for app in dataset_android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) # make sure this is inside the if block

In [77]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


Let's remove non-english apps now from our partially
cleaned dataset 'android-clean'.

In [78]:
print(android_clean[4412][0])

中国語 AQリスニング


In [79]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127: # to account for
                                 # emojis and symbols
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

In [80]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


In [81]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
True


That's good. The two apps tested in the cell above are
directed to english speakers.

In [82]:
android_english = []
ios_english = []
# We will be cleaning the non english apps in both the
# android dataset and the ios apps

In [84]:
for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in dataset_iOs:
    name = app[1]
    if is_english(name):
        ios_english.append(app)

### Isolating the free apps from the purchase store

In [85]:
android_final = []
ios_final = []

In [86]:
for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)
        
for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)
        
print(len(android_final))
print(len(ios_final))

8864
3222


Since the revenue is generated by the in apps-adds.
We want to fous on the kind of apps that are most commonly downloaded.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

Build a minimal Android version of the app, and add it to Google Play.
If the app has a good response from users, we develop it further.
If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.


Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets.
For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

### Most common apps by genre

we'll build a frequency table for the prime_genre column of the App Store data set and the Genres and Category columns of the Google Play data set.

In [87]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages

The following function will display the percentages in
the descending order.

In [88]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0], '%')

In [89]:
display_table(ios_final, -5)

Games : 58.16263190564867 %
Entertainment : 7.883302296710118 %
Photo & Video : 4.9658597144630665 %
Education : 3.662321539416512 %
Social Networking : 3.2898820608317814 %
Shopping : 2.60707635009311 %
Utilities : 2.5139664804469275 %
Sports : 2.1415270018621975 %
Music : 2.0484171322160147 %
Health & Fitness : 2.0173805090006205 %
Productivity : 1.7380509000620732 %
Lifestyle : 1.5828677839851024 %
News : 1.3345747982619491 %
Travel : 1.2414649286157666 %
Finance : 1.1173184357541899 %
Weather : 0.8690254500310366 %
Food & Drink : 0.8069522036002483 %
Reference : 0.5586592178770949 %
Business : 0.5276225946617008 %
Book : 0.4345127250155183 %
Navigation : 0.186219739292365 %
Medical : 0.186219739292365 %
Catalogs : 0.12414649286157665 %


'Games' genre dominate the datset of free apps. The second to 'Games' is 'Entertainment' category.

This represents the number of apps dominant in the App Store which is different from ranking the apps genre by number of downloads they get which is a true reflection of the popularity of the apps.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [90]:
display_table(android_final, 1) # Category

FAMILY : 18.907942238267147 %
GAME : 9.724729241877256 %
TOOLS : 8.461191335740072 %
BUSINESS : 4.591606498194946 %
LIFESTYLE : 3.9034296028880866 %
PRODUCTIVITY : 3.892148014440433 %
FINANCE : 3.7003610108303246 %
MEDICAL : 3.531137184115524 %
SPORTS : 3.395758122743682 %
PERSONALIZATION : 3.3167870036101084 %
COMMUNICATION : 3.2378158844765346 %
HEALTH_AND_FITNESS : 3.0798736462093865 %
PHOTOGRAPHY : 2.944494584837545 %
NEWS_AND_MAGAZINES : 2.7978339350180503 %
SOCIAL : 2.6624548736462095 %
TRAVEL_AND_LOCAL : 2.33528880866426 %
SHOPPING : 2.2450361010830324 %
BOOKS_AND_REFERENCE : 2.1435018050541514 %
DATING : 1.861462093862816 %
VIDEO_PLAYERS : 1.7937725631768955 %
MAPS_AND_NAVIGATION : 1.3989169675090252 %
FOOD_AND_DRINK : 1.2409747292418771 %
EDUCATION : 1.1620036101083033 %
ENTERTAINMENT : 0.9589350180505415 %
LIBRARIES_AND_DEMO : 0.9363718411552346 %
AUTO_AND_VEHICLES : 0.9250902527075812 %
HOUSE_AND_HOME : 0.8235559566787004 %
WEATHER : 0.8009927797833934 %
EVENTS : 0.710740072

In Google Playstore, 'Family' apps dominate in number.

Let us see the 'Genre' column of the apps in the Google Play store.

In [91]:
display_table(android_final, -4)

Tools : 8.449909747292418 %
Entertainment : 6.069494584837545 %
Education : 5.347472924187725 %
Business : 4.591606498194946 %
Productivity : 3.892148014440433 %
Lifestyle : 3.892148014440433 %
Finance : 3.7003610108303246 %
Medical : 3.531137184115524 %
Sports : 3.463447653429603 %
Personalization : 3.3167870036101084 %
Communication : 3.2378158844765346 %
Action : 3.1024368231046933 %
Health & Fitness : 3.0798736462093865 %
Photography : 2.944494584837545 %
News & Magazines : 2.7978339350180503 %
Social : 2.6624548736462095 %
Travel & Local : 2.3240072202166067 %
Shopping : 2.2450361010830324 %
Books & Reference : 2.1435018050541514 %
Simulation : 2.0419675090252705 %
Dating : 1.861462093862816 %
Arcade : 1.8501805054151623 %
Video Players & Editors : 1.7712093862815883 %
Casual : 1.7599277978339352 %
Maps & Navigation : 1.3989169675090252 %
Food & Drink : 1.2409747292418771 %
Puzzle : 1.128158844765343 %
Racing : 0.9927797833935018 %
Role Playing : 0.9363718411552346 %
Libraries & D

'Genre' column is more granular and the distribution of the apps is less skewed in comparison to the 'Category' column of the Google Playstore.

### Most "popular" apps by Genre in the datasets

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

In [92]:
genres_ios = freq_table(ios_final, -5)
# obtain the frequency table of the genre of the apps

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Utilities : 18684.456790123455
Health & Fitness : 23298.015384615384
Book : 39758.5
News : 21248.023255813954
Productivity : 21028.410714285714
Lifestyle : 16485.764705882353
Finance : 31467.944444444445
Education : 7003.983050847458
Sports : 23008.898550724636
Weather : 52279.892857142855
Games : 22788.6696905016
Navigation : 86090.33333333333
Medical : 612.0
Reference : 74942.11111111111
Shopping : 26919.690476190477
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Travel : 28243.8
Music : 57326.530303030304
Photo & Video : 28441.54375
Business : 7491.117647058823
Catalogs : 4004.0
Social Networking : 71548.34905660378


Notice that on average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [93]:
for app in ios_final:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Apps like Facebook, Pinterestm Skype have huge contributions in average number calculated above for the Social Networking apps. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold.

If these apps are reomved, we can have a more balanced data on the avergae number of installs per app genre for each genre. This is for a more fine-grained analysis.

Now we turn to Google Playstore

In the column index 5, we have the number of installs against each app.

In [94]:
display_table(android_final, 5)

1,000,000+ : 15.726534296028879 %
100,000+ : 11.552346570397113 %
10,000,000+ : 10.548285198555957 %
10,000+ : 10.198555956678701 %
1,000+ : 8.393501805054152 %
100+ : 6.915613718411552 %
5,000,000+ : 6.825361010830325 %
500,000+ : 5.561823104693141 %
50,000+ : 4.7721119133574 %
5,000+ : 4.512635379061372 %
10+ : 3.5424187725631766 %
500+ : 3.2490974729241873 %
50,000,000+ : 2.3014440433213 %
100,000,000+ : 2.1322202166064983 %
50+ : 1.917870036101083 %
5+ : 0.78971119133574 %
1+ : 0.5076714801444043 %
500,000,000+ : 0.2707581227436823 %
1,000,000,000+ : 0.22563176895306858 %
0+ : 0.04512635379061372 %
0 : 0.01128158844765343 %


Interpreting the above results:
Apps 1,000,000+ installs comprise 15.72% of the apps in the dataset.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [95]:
categories_android = freq_table(android_final, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_final:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

TOOLS : 10801391.298666667
EVENTS : 253542.22222222222
MAPS_AND_NAVIGATION : 4056941.7741935486
SHOPPING : 7036877.311557789
SPORTS : 3638640.1428571427
PRODUCTIVITY : 16787331.344927534
FOOD_AND_DRINK : 1924897.7363636363
PARENTING : 542603.6206896552
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
EDUCATION : 1833495.145631068
FINANCE : 1387692.475609756
BEAUTY : 513151.88679245283
WEATHER : 5074486.197183099
MEDICAL : 120550.61980830671
DATING : 854028.8303030303
SOCIAL : 23253652.127118643
AUTO_AND_VEHICLES : 647317.8170731707
TRAVEL_AND_LOCAL : 13984077.710144928
COMMUNICATION : 38456119.167247385
HOUSE_AND_HOME : 1331540.5616438356
BOOKS_AND_REFERENCE : 8767811.894736841
PERSONALIZATION : 5201482.6122448975
FAMILY : 3695641.8198090694
ENTERTAINMENT : 11640705.88235294
COMICS : 817657.2727272727
BUSINESS : 1712290.1474201474
LIBRARIES_AND_DEMO : 638503.734939759
PHOTOGRAPHY : 17840110.40229885
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_MAGAZINES : 9549178.467741935
HEALT

On average, communication apps have the most installs: 38,456,119.

Once again, this number could be heavily skewed by the app like Facebook, Skype, Chrome.

In [96]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

It seems like there is a good number of apps in 'COMMUNICATION' category with 100,000,000+ installs.

Let's explore another category: BOOKS_AND_REFERENCE

In [99]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The category seems to cover a lot many types of apps like ebooks, dictionaries and libraries.

Let's see which apps skew the number of intalls of this category.

In [100]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


In [108]:
print(android_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [110]:
unique_categories =[]

for app in android_final:
    category = app[1]
    if category not in unique_categories:
        unique_categories.append(category)

In [111]:
print(unique_categories)

['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE', 'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL', 'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL', 'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER', 'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION']


Let's analyze 'NEWS_AND_MAGAZINES'

In [112]:
for app in android_final:
    if app[1] == 'NEWS_AND_MAGAZINES' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Twitter : 500,000,000+
Flipboard: News For Our Time : 500,000,000+
Google News : 1,000,000,000+


Twitter is listed as 'NEWS_AND_MAGAZINES' category which also serves the purpose of social networking and for this reason, receives a huge number of downloads. This is important to highlight because the average number of downloads for this category is 9549178.46 which reflects how much social media is used as a news service to public.